## Predicting heart disease using machine learning

This notebook looks into using various python-based machine learning and data science
libraries in an attempt to build a machine learning model capable of predicting
whether or not sosmeone has heart on their medical attributes.
we're going to take the following approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modeling
6. Experimentaion
## 1. Problem Defination